# EECE 5554: A Sample ROS Publisher

Let's take a look at a sample Python script that generates a publisher node in ROS. If you try to run this code, it will fail, because Jupyter notebooks don't inherently know about ROS Python modules.

In [ ]:
# -*- coding: utf-8 -*-

import rospy
import serial
from math import sin, pi
from std_msgs.msg import Float64
from nav_msgs.msg import Odometry

def paro_to_depth(pressure, latitude):
    '''
    Given pressure (in m fresh) and latitude (in radians) returns ocean depth (in m.).  Uses the formula discovered and presented by Leroy and Parthiot in: Claude C. Leroy and Francois Parthiot, 'Depth-pressure relationships in the oceans and seas', J. Acoustic Society of America, March 1998, p1346-.
    '''
    # Convert the input m/fw into MPa, as the equation expects MPa.
    pressure = pressure * 0.0098066493
    # Gravity at Latitude.
    g = 9.780318 * (1 + 5.2788e-3*sin(latitude)**2 -
                        2.36e-5*sin(latitude)**4)
    # Now calculate the 'standard ocean' depth.
    Zs_num = (9.72659e2*pressure - 2.512e-1*pressure**2 +
              2.279e-4*pressure**3 - 1.82e-7*pressure**4)
    Zs_den = g + 1.092e-4*pressure
    return Zs_num / Zs_den

if __name__ == '__main__':
    SENSOR_NAME = "paro"
    rospy.init_node('depth_paro')
    serial_port = rospy.get_param('~port','/dev/ttyS1')
    serial_baud = rospy.get_param('~baudrate',9600)
    sampling_rate = rospy.get_param('~sampling_rate',5.0)
    offset = rospy.get_param('~atm_offset',12.121) # in meter ??
    latitude_deg = rospy.get_param('~latitude',41.526)  # deg 41.526 N is Woods Hole
  
    port = serial.Serial(serial_port, serial_baud, timeout=3.)
    rospy.logdebug("Using depth sensor on port "+serial_port+" at "+str(serial_baud))
    rospy.logdebug("Using latitude = "+str(latitude_deg)+" & atmosphere offset = "+str(offset))
    rospy.logdebug("Initializing sensor with *0100P4\\r\\n ...")
    
    sampling_count = int(round(1/(sampling_rate*0.007913)))
    port.write('*0100EW*0100PR='+str(sampling_count)+'\r\n')  # cmd from 01 to 00 to set sampling period
    rospy.sleep(0.2)        
    line = port.readline()
    port.write('*0100P4\r\n')  # cmd from 01 to 00 to sample continuously
    
    latitude = latitude_deg * pi / 180.
    depth_pub = rospy.Publisher(SENSOR_NAME+'/depth', Float64, queue_size=5)
    pressure_pub = rospy.Publisher(SENSOR_NAME+'/pressure', Float64, queue_size=5)
    odom_pub = rospy.Publisher(SENSOR_NAME+'/odom',Odometry, queue_size=5)
    
    rospy.logdebug("Initialization complete")
    
    rospy.loginfo("Publishing pressure and depth.")
    
    odom_msg = Odometry()
    odom_msg.header.frame_id = "odom"
    odom_msg.child_frame_id = SENSOR_NAME
    odom_msg.header.seq=0
    
    sleep_time = 1/sampling_rate - 0.025
   
    try:
        while not rospy.is_shutdown():
            line = port.readline()
            #print line
            if line == '':
                rospy.logwarn("DEPTH: No data")
            else:
                if line.startswith('*0001'):
                    odom_msg.header.stamp=rospy.Time.now()                
                    try: pressure = float(line[5:].strip())
                    except: 
                        rospy.logwarn("Data exception: "+line)
                        continue
                pressure_pub.publish(pressure)
                depth_mes = paro_to_depth(pressure - offset, latitude_deg)
                depth_pub.publish(depth_mes)
                odom_msg.pose.pose.position.z = -depth_mes
                odom_msg.header.seq+=0
                odom_pub.publish(odom_msg)
            rospy.sleep(sleep_time)
            
    except rospy.ROSInterruptException:
        port.close()
    
    except serial.serialutil.SerialException:
        rospy.loginfo("Shutting down paro_depth node...")

That's a lot at once, so let's break it down into its pieces.

### Taking it a few lines at a time

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

The first line is called the shebang, and you need it to tell Linux that it is a Python script. The second line specifies the character encoding.

In [ ]:
import serial
from math import sin, pi
import rospy
from std_msgs.msg import Float64
from nav_msgs.msg import Odometry

These lines import specific ROS and Python modules we will need to run the code. 

In [4]:
def paro_to_depth(pressure, latitude):
    '''
    Given pressure (in m fresh) and latitude (in radians) returns ocean depth (in m.).  
    Uses the formula discovered and presented by Leroy and Parthiot in: Claude C. Leroy and Francois Parthiot, 
    'Depth-pressure relationships in the oceans and seas', J. Acoustic Society of America, March 1998, p1346-.
    '''
    pressure_MPa = pressure_mfw * 0.0098066493
    gravity_at_latitude = 9.780318 * (1 + 5.2788e-3*sin(latitude)**2 -
                        2.36e-5*sin(latitude)**4) #corrects for the effects of latitutde on gravity
    std_ocean_depth = (9.72659e2*pressure_MPa - 2.512e-1*pressure_MPa**2 +
              2.279e-4*pressure_MPa**3 - 1.82e-7*pressure_MPa**4)
    density = gravity_at_latitude + 1.092e-4*pressure_MPa
    return std_ocean_depth / density

This function converts latitude and pressure to an ocean depth. Here, we see single-line comments with #, multiline comments with ''' ''', and multi-line code. Python uses <code>**</code> instead of <code>^</code> for exponents.

When you're writing code, don't forget to give good, specific variable names!

In [ ]:
if __name__ == '__main__':
    SENSOR_NAME = "paro"
    rospy.init_node('depth_paro')

The code underneath <code> if __name__ == '__main__': </code> is only executed if this file is run directly by the Python interpreter rather than being called from another script. Remember that Python is really, really specific about white space, so you need to have the same number of spaces (or tabs) for each block of code.

The second line initializes the publisher node with name 'depth_paro'. You can only have one node in a rospy process, so you can only call rospy.init_node() once. 

In [ ]:
    serial_port = rospy.get_param('~port','/dev/ttyS1')
    serial_baud = rospy.get_param('~baudrate',9600)
    sampling_rate = rospy.get_param('~sampling_rate',5.0)
    offset = rospy.get_param('~atm_offset',12.121) # in meter ??
    latitude_deg = rospy.get_param('~latitude',41.526)  # deg 41.526 N is Woods Hole

Here, the ROS parameter server is being queried for the values of the serial port address, the baud rate (communication speed between computer and sensor), the sampling rate, the ATM offset, and the latitude. If those values are not set in the parameter server, the values specified with the second argument are used. More here on parameter servers: https://wiki.ros.org/Parameter%20Server

In [ ]:
    rospy.logdebug("Using depth sensor on port " + serial_port + " at " + str(serial_baud))
    rospy.logdebug("Using latitude = " + str(latitude_deg) + " & atmosphere offset = " + str(offset))
    rospy.logdebug("Initializing sensor with *0100P4\\r\\n ...")

These lines post log debugging information as strings to the terminal and to the topic /rosout. More here: https://wiki.ros.org/rospy/Overview/Logging

In [ ]:
    port = serial.Serial(serial_port, serial_baud, timeout=3.)
    
    sampling_count = int(round(1/(sampling_rate*0.007913)))
    port.write('*0100EW*0100PR='+str(sampling_count)+'\r\n')  # cmd from 01 to 00 to set sampling period
    rospy.sleep(0.2)        
    line = port.readline()
    port.write('*0100P4\r\n')  # cmd from 01 to 00 to sample continuously

__[PySerial's](https://pythonhosted.org/pyserial/)__ <code>serial.Serial</code> is set with the port address to be read from/to, the baud rate, and how long to wait before timing out.
The script calculates how many samples are required based on the sampling rate, writes a string to the port (sending information to the sensor), waits for 0.2 seconds, reads the sensor's reply, and writes another string. We're still in setup here.

In [ ]:
    latitude = latitude_deg * pi / 180.
    depth_pub = rospy.Publisher(SENSOR_NAME+'/depth', Float64, queue_size=5)
    pressure_pub = rospy.Publisher(SENSOR_NAME+'/pressure', Float64, queue_size=5)
    odom_pub = rospy.Publisher(SENSOR_NAME+'/odom',Odometry, queue_size=5)
    
    rospy.logdebug("Initialization complete")
    
    rospy.loginfo("Publishing pressure and depth.")

Now we're creating handles for the messages we want to publish. Two publisher handles are set up, one that will publish to the topic /depth and the other to the topic /pressure. Both publishers will publish Float64 messages.

Then an Odometry handle is set up to publish to topic /odom. We'll get to odometry messages later in the course.

Finally, more info and debugging messages that will be automagically published to /rosout.

In [ ]:
    odom_msg = Odometry()
    odom_msg.header.frame_id = "odom"
    odom_msg.child_frame_id = SENSOR_NAME
    odom_msg.header.seq=0
    
    sleep_time = 1/sampling_rate - 0.025

The script calls <code>Odometry()</code>, which creates an Odometry message with all the appropriate fields. The header.frame_id is specified as "odom", the child_frame_id is the sensor name set earlier in the code, and the header.seq is 0. We'll talk more about header messages in class.

At this point, the code to read and write information to the serial port is set, the publisher node has been initialized, and we have handles to publish pressure, depth, and odometry. We are ready to start publishing messages once they're read from the sensor.

In [ ]:
   try:
        while not rospy.is_shutdown():
            line = port.readline()
            if line == '':
                rospy.logwarn("DEPTH: No data")
            else:
                if line.startswith('*0001'):
                    odom_msg.header.stamp=rospy.Time.now()                
                    try: pressure = float(line[5:].strip())
                    except: 
                        rospy.logwarn("Data exception: "+line)
                        continue
                pressure_pub.publish(pressure)
                depth_mes = paro_to_depth(pressure - offset, latitude_deg)
                depth_pub.publish(depth_mes)
                odom_msg.pose.pose.position.z = -depth_mes
                odom_msg.header.seq+=0
                odom_pub.publish(odom_msg)
            rospy.sleep(sleep_time)

Finally, the script tries to execute the code underneath <code> try:</code>. 
The line <code> while not rospy.is_shutdown():</code> loops infinitely while ROS is running, continuing to publish messages at the frequency specified by <code>rospy.sleep(sleep_time)</code>. 

If an empty string is read from the serial port (the sensor), a warning string will be published. Otherwise, the header will be updated with the current system time, the pressure information will be published, the pressure will be converted to depth, and the depth information will also be published. Finally, the pose and header of the odometry message will be set.

If an exception is thrown, it will take the action specified under the exceptions.

In [ ]:
    except rospy.ROSInterruptException:
        port.close()
    
    except serial.serialutil.SerialException:
        rospy.loginfo("Shutting down paro_depth node...")